In [ ]:
import time
import RPi.GPIO as GPIO
from time import sleep
from picamera2 import Picamera2, Preview
from picamera2.controls import Controls
import numpy as np
import matplotlib.pyplot as plt
import os
import tifffile
from datetime import datetime
import tkinter as tk


In [ ]:
picam2 = Picamera2()
led = 23
GPIO.setmode(GPIO.BCM)
GPIO.setup(led, GPIO.OUT)

In [ ]:
# Set camera controls
controls = {"ExposureTime": 4000000, #microseconds
            "AnalogueGain":1.0, # 1 = ISO 100
            "AeEnable": False, # Auto exposure and Gain
            "AwbEnable": False,# Auto white Balance
            "FrameDurationLimits": (114,239000000)} #Min/Max frame duration

# Setup config parameters
preview_config = picam2.create_preview_configuration(raw={"size": picam2.sensor_resolution, "format": "SBGGR12",},
                                                     controls = controls) 

In [ ]:
def capture_raw(suffix, num_images):
     global raw_crop
    
     for i in range(num_images):
        picam2.start_preview(Preview.QTGL)
        picam2.configure(preview_config)

        picam2.start() # Start Camera
    
        time.sleep(2)

        GPIO.output(led, GPIO.HIGH) # Turn on LED
        time.sleep(1)

        #Capture image in unpacked RAW format 12bit dynamic range (16bit array)
        raw = picam2.capture_array("raw").view(np.uint16)

        time.sleep(1)
        GPIO.output(led, GPIO.LOW) # Turn off LED

        print(picam2.capture_metadata())

        picam2.stop_preview()
        picam2.stop()

        raw_crop = raw[0:3040, 0:4056] # Remove padding from each row of pixels

        base_filename = "RAW"
        save_dir = '/home/martin/Desktop/Images/'

        # Create a new folder with date stamp if it does not exist
        date_str = datetime.now().strftime("%Y-%m-%d")
        save_dir = os.path.join(save_dir, f'{base_filename}_{date_str}')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        # Construct the filename with the user-defined suffix
        #filename = f'{base_filename}_{date_str}_air_sat_{suffix}.tiff'

        # Check if the filename with the user-defined suffix already exists in the folder
        filename = f'{base_filename}_{date_str}_air_sat{suffix}.tiff'
        count = 1
        while os.path.exists(os.path.join(save_dir, filename)):
            # If the filename exists, add a number to the suffix and try again
            filename = f'{base_filename}_{date_str}_air_sat{suffix}_{count}.tiff'
            count += 1

        # Save the image with the updated filename
        tifffile.imwrite(os.path.join(save_dir, filename), raw_crop)
        print("RAW Image Saved")
   
#Create a function to get the value of the suffix from the entry field and call the capture_raw function
def capture_image():
    suffix_value = suffix_entry.get()
    num_images = int(num_images_entry.get())
    capture_raw(suffix_value, num_images)


# Create the Tkinter window and add the button and entry field
window = tk.Tk()
window.title("Capture RAW Image")
window.geometry("300x200")
suffix_label = tk.Label(window, text="Enter Suffix Value:")
suffix_label.pack()
suffix_entry = tk.Entry(window)
suffix_entry.pack()
capture_button = tk.Button(window, text="Capture Raw Image", command=capture_image)
capture_button.pack()
num_images_label = tk.Label(window, text="Enter Number of Images:")
num_images_label.pack()
num_images_entry = tk.Entry(window)
num_images_entry.pack()
window.mainloop()

In [ ]:
  # Plot the histogram
  plt.clf()
  center_crop = raw_crop[1000:2000, 2000:3000]
  # Get color channels in bayer order (BGGR)
  red = center_crop[1::2,1::2]
  green1 = center_crop[0::2,1::2]
  green2 = center_crop[1::2,0::2]
  green = np.add(green1,green2)/2
  # Make histogram for red and green channel
  Colors = ("red","green")
  Channel_ids = (red,green)
  for channel_id, c in zip(Channel_ids,Colors):
      histogram, bin_edges = np.histogram(channel_id, bins=4095, range=(0,4095))
      plt.plot(bin_edges[0:-1], histogram, color=c)
  plt.title("Red_Green histogram")
  plt.xlabel("Pixel intensity")
  plt.ylabel("Pixel Frequency")
  plt.draw()
  # Display the image
  plt.figure()
  plt.imshow(raw_crop, cmap="gray")
  plt.show()
    